In [0]:
import pyspark
import pandas as pd
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sc

<SparkContext master=local[8] appName=Databricks Shell>

In [0]:
f = spark.read.csv('dbfs:/FileStore/tables/urls.csv' , header=True)    # Loading the data into table

In [0]:
from bs4 import *
from requests import *

def get_html(url):
    try:
        req = get(url)
        soup = BeautifulSoup(req.text , 'html.parser')
    except Exception:
        print('Request Not Sent') 
       
    try:
        contents = soup.find('div' , attrs={'id':'bodyContent'})
        bio_content = contents.find('table' , class_ = 'infobox biography vcard')
    except Exception:
        print('Data Not Loaded')

    # Name , dob , native details seperated.

    try:
        # To change all labels in 'label' list and all data in 'details' list
        name = bio_content.find('div' , class_ = 'nickname').text
        b_day = bio_content.find('span' , class_ = 'bday').text
        birth_place = bio_content.find('div' , class_ = 'birthplace').text
    except Exception:
        print("Error")
    return 'Name: ' + name + '  ' + 'Birth Date: ' + b_day + '  ' + ' Birth Place: ' + birth_place

In [0]:
updated_df = f.withColumnRenamed('Base Urls' , 'url')      # Re-naming the existing column
updated_df.collect()

[Row(url='https://en.wikipedia.org/wiki/Bill_Gates'),
 Row(url='https://en.wikipedia.org/wiki/Elon_Musk'),
 Row(url='https://en.wikipedia.org/wiki/Sundar_Pichai'),
 Row(url='https://en.wikipedia.org/wiki/Satya_Nadella')]

In [0]:
from pyspark.sql.functions import col, udf          # converting the python UDF to SQL in-built fn.
from pyspark.sql.types import StringType

get_html1 = udf(get_html , StringType())

In [0]:
updated_df.select(col('url') , get_html1(col('url')).alias('Details')).show(truncate=False)

+-------------------------------------------+----------------------------------------------------------------------------------------------------+
|url                                        |Details                                                                                             |
+-------------------------------------------+----------------------------------------------------------------------------------------------------+
|https://en.wikipedia.org/wiki/Bill_Gates   |Name: William Henry Gates III  Birth Date: 1955-10-28   Birth Place: Seattle, Washington, U.S.      |
|https://en.wikipedia.org/wiki/Elon_Musk    |Name: Elon Reeve Musk  Birth Date: 1971-06-28   Birth Place: Pretoria, Transvaal, South Africa      |
|https://en.wikipedia.org/wiki/Sundar_Pichai|Name: Pichai Sundararajan  Birth Date: 1972-06-10   Birth Place: Madurai, Tamil Nadu, India         |
|https://en.wikipedia.org/wiki/Satya_Nadella|Name: Satya Narayana Nadella  Birth Date: 1967-08-19   Birth Place: Hyder